In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [8]:
import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Set to the Spark root directory
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("IntrusionDetection") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/08 14:36:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Set to the Spark root directory
from pyspark.sql import SparkSession

# Initialize Spark session with specific resource limits
spark = SparkSession.builder \
    .appName("IntrusionDetection") \
    .config("spark.executor.memory", "3g")  # Amount of memory to use per executor process
    .config("spark.executor.cores", "1")  # Number of cores to use per executor
    .config("spark.driver.memory", "3g")  # Amount of memory to use for the driver process
    .config("spark.driver.cores", "1")  # Number of cores to use for the driver process
    .config("spark.cores.max", "2")  # Maximum total cores to allocate for the application
    .getOrCreate()


IndentationError: unexpected indent (1859829144.py, line 9)

In [9]:
# Define the path to the dataset in HDFS
hdfs_path = "hdfs://localhost:54310/datasets/NF-UQ-NIDS-v2.csv"  # Adjust the filename if necessary
# Define the path to the dataset in HDFS
#hdfs_path = "hdfs:///datasets/NF-UQ-NIDS-v2/dataset.csv"  # Adjust the filename if necessary


In [10]:
df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

In [12]:
# Display the schema of the DataFrame
df.printSchema()

# Show summary statistics of the dataset
df.describe().show()


root
 |-- IPV4_SRC_ADDR: string (nullable = true)
 |-- L4_SRC_PORT: integer (nullable = true)
 |-- IPV4_DST_ADDR: string (nullable = true)
 |-- L4_DST_PORT: integer (nullable = true)
 |-- PROTOCOL: integer (nullable = true)
 |-- L7_PROTO: double (nullable = true)
 |-- IN_BYTES: integer (nullable = true)
 |-- IN_PKTS: integer (nullable = true)
 |-- OUT_BYTES: integer (nullable = true)
 |-- OUT_PKTS: integer (nullable = true)
 |-- TCP_FLAGS: integer (nullable = true)
 |-- CLIENT_TCP_FLAGS: integer (nullable = true)
 |-- SERVER_TCP_FLAGS: integer (nullable = true)
 |-- FLOW_DURATION_MILLISECONDS: integer (nullable = true)
 |-- DURATION_IN: integer (nullable = true)
 |-- DURATION_OUT: integer (nullable = true)
 |-- MIN_TTL: integer (nullable = true)
 |-- MAX_TTL: integer (nullable = true)
 |-- LONGEST_FLOW_PKT: integer (nullable = true)
 |-- SHORTEST_FLOW_PKT: integer (nullable = true)
 |-- MIN_IP_PKT_LEN: integer (nullable = true)
 |-- MAX_IP_PKT_LEN: integer (nullable = true)
 |-- SRC_TO

24/11/07 19:58:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+------------------+-------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+--------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+------------------+--------+---------------+
|summary|IPV4_SRC_ADDR|       L4_SRC_PORT|IPV4_DST_ADDR|     

In [11]:
# Import necessary functions from PySpark
from pyspark.sql.functions import col, sum as spark_sum, countDistinct

# Check for missing values in each column
missing_values = df.select([(spark_sum(col(c).isNull().cast("int")).alias(c)) for c in df.columns])
missing_values.show()

# Get the unique count of values for each column
unique_counts = df.agg(*[countDistinct(col(c)).alias(c) for c in df.columns])
unique_counts.show()


24/11/08 14:42:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-----------+-------------+-----------+--------+--------+--------+-------+---------+--------+---------+----------------+----------------+--------------------------+-----------+------------+-------+-------+----------------+-----------------+--------------+--------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+--------------+---------------+---------+--------------+------------+--------------+--------------+--------------------+-----+------+-------+
|IPV4_SRC_ADDR|L4_SRC_PORT|IPV4_DST_ADDR|L4_DST_PORT|PROTOCOL|L7_PROTO|IN_BYTES|IN_PKTS|OUT_BYTES|OUT_PKTS|TCP_FLAGS|CLIENT_TCP_FLAGS|SERVER_TCP_FLAGS|FLOW_DURATION_MILLISECONDS|DURATION_IN|DURATION_OUT|MIN_TTL|MAX_TTL|LONGEST_FLOW_PKT|SHORTEST_FLOW_

+-------------+-----------+-------------+-----------+--------+--------+--------+-------+---------+--------+---------+----------------+----------------+--------------------------+-----------+------------+-------+-------+----------------+-----------------+--------------+--------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+--------------+---------------+---------+--------------+------------+--------------+--------------+--------------------+-----+------+-------+
|IPV4_SRC_ADDR|L4_SRC_PORT|IPV4_DST_ADDR|L4_DST_PORT|PROTOCOL|L7_PROTO|IN_BYTES|IN_PKTS|OUT_BYTES|OUT_PKTS|TCP_FLAGS|CLIENT_TCP_FLAGS|SERVER_TCP_FLAGS|FLOW_DURATION_MILLISECONDS|DURATION_IN|DURATION_OUT|MIN_TTL|MAX_TTL|LONGEST_FLOW_PKT|SHORTEST_FLOW_

In [ ]:
from pyspark.sql.functions import col

# Define categorical and continuous features based on unique value counts and feature characteristics
categorical_features = [
    "PROTOCOL", "TCP_FLAGS", "CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", 
    "ICMP_TYPE", "ICMP_IPV4_TYPE", "DNS_QUERY_ID", "DNS_QUERY_TYPE", 
    "Label", "Attack", "Dataset"
]

continuous_features = [
    "L4_SRC_PORT", "L4_DST_PORT", "IN_BYTES", "IN_PKTS", "OUT_BYTES", 
    "OUT_PKTS", "FLOW_DURATION_MILLISECONDS", "DURATION_IN", "DURATION_OUT", 
    "MIN_TTL", "MAX_TTL", "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", 
    "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", 
    "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS", 
    "RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS", "SRC_TO_DST_AVG_THROUGHPUT", 
    "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", 
    "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES", 
    "TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT", "FTP_COMMAND_RET_CODE"
]

# Show summary statistics for continuous features
df.select([col(c) for c in continuous_features]).describe().show()

# Show unique value counts for categorical features
for feature in categorical_features:
    df.select(feature).distinct().show(truncate=False)
